In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [6]:
def retrieve_stock_data(stock_symbol, start_date, end_date):
    data = yf.download(stock_symbol, start = start_date, end = end_date)
    return data

In [8]:
stock_data = retrieve_stock_data("APPL", "2020-01-01", "2023-01-01")

Failed to get ticker 'APPL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1002)')))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['APPL']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [ ]:
stock_data.head()